In [1]:
from songs import *

In [11]:
def addAllTransposedVersions(Songs, song):
    while song.transpose(1):
        pass

    Songs.append(song.notes)
    while song.transpose(-1):
        Songs.append(song.notes)
Songs = []
for i in range(1, 35):
    addAllTransposedVersions(Songs, Song('basic midi/track (' + str(i) + ').mid'))
Songs = np.array(Songs)

ERROR! out of range!
ERROR! out of range!
ERROR! out of range!


In [14]:
def NextBatch():
    return (Songs[np.random.randint(0, len(Songs), 128)].reshape(128, -1), 
            Songs[np.random.randint(0, len(Songs), 128)].reshape(128, -1))

In [18]:
import tensorflow as tf
import numpy as np

# Training Parameters
learning_rate = 0.01
num_steps = 10000
batch_size = 128

display_step = 100
#examples_to_show = 10

# Network Parameters
num_hidden_1 = 10*128 # 1st layer num features
num_hidden_2 = 5*128 # 2nd layer num features
num_input = 13*128 # MNIST data input (img shape: 28*28)

# tf Graph input (only pictures)
X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_input])

weights = {
    'encoder_h1': tf.Variable(tf.random_normal([num_input, num_hidden_1])),
    'encoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_hidden_2])),
    'decoder_h1': tf.Variable(tf.random_normal([num_hidden_2, num_hidden_1])),
    'decoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_input])),
}
biases = {
    'encoder_b1': tf.Variable(tf.random_normal([num_hidden_1])),
    'encoder_b2': tf.Variable(tf.random_normal([num_hidden_2])),
    'decoder_b1': tf.Variable(tf.random_normal([num_hidden_1])),
    'decoder_b2': tf.Variable(tf.random_normal([num_input])),
}

# Building the encoder
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']),
                                   biases['encoder_b1']))
    # Encoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']),
                                   biases['encoder_b2']))
    return layer_2


# Building the decoder
def decoder(x):
    # Decoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']),
                                   biases['decoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']),
                                   biases['decoder_b2']))
    return layer_2

# Construct model
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Prediction
y_pred = decoder_op

# Define loss and optimizer, minimize the squared error
loss = tf.reduce_mean(tf.pow(tf.reduce_mean(tf.pow(Y - y_pred, 2), 1), 0.25))
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(loss)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# Start Training
# Start a new TF session
sess = tf.Session()

# Run the initializer
sess.run(init)

# Training
for i in range(1, num_steps+1):
    # Prepare Data
    # Get the next batch of MNIST data (only images are needed, not labels)
    batch_x, batch_y = NextBatch()

    # Run optimization op (backprop) and cost op (to get loss value)
    _, l = sess.run([optimizer, loss], feed_dict={X: batch_x, Y: batch_y})
    # Display logs per step
    if i % display_step == 0 or i == 1:
        print('Step %i: Minibatch Loss: %f' % (i, l))

Step 1: Minibatch Loss: 0.830349
Step 100: Minibatch Loss: 0.826200
Step 200: Minibatch Loss: 0.646676
Step 300: Minibatch Loss: 0.559936
Step 400: Minibatch Loss: 0.555877
Step 500: Minibatch Loss: 0.555277
Step 600: Minibatch Loss: 0.551012
Step 700: Minibatch Loss: 0.552359
Step 800: Minibatch Loss: 0.553288
Step 900: Minibatch Loss: 0.550889
Step 1000: Minibatch Loss: 0.551866
Step 1100: Minibatch Loss: 0.551995
Step 1200: Minibatch Loss: 0.547676
Step 1300: Minibatch Loss: 0.549034
Step 1400: Minibatch Loss: 0.549445
Step 1500: Minibatch Loss: 0.546668
Step 1600: Minibatch Loss: 0.548788
Step 1700: Minibatch Loss: 0.548210
Step 1800: Minibatch Loss: 0.547761
Step 1900: Minibatch Loss: 0.548616
Step 2000: Minibatch Loss: 0.545189
Step 2100: Minibatch Loss: 0.546429
Step 2200: Minibatch Loss: 0.547509
Step 2300: Minibatch Loss: 0.547606
Step 2400: Minibatch Loss: 0.548274
Step 2500: Minibatch Loss: 0.545561
Step 2600: Minibatch Loss: 0.547859
Step 2700: Minibatch Loss: 0.547895
Step

In [20]:
batch_x, _ = NextBatch()

In [22]:
batch_x[0].reshape(128, 13)

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,

In [44]:
trans = sess.run([y_pred], feed_dict={X: batch_x})
trans = trans[0]

In [ ]:
MySong(batch_x[3].reshape(128, 13)).play()

In [52]:
MySong((trans[4].reshape(128, 13) > 0.5).astype(int)).play()